In [ ]:
from fortigate_methods import FortigateMethods
import requests
import logging
import urllib3
import sys
import os
sys.path.append(os.path.abspath('../vault'))
from vault_methods import VaultMethods

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

logging.basicConfig(format='%(asctime)s %(message)s',
                    datefmt='%m/%d/%Y %I:%M:%S %p',
                    level=logging.INFO)

if __name__ == '__main__':
    vault_session = VaultMethods()
    vault_session.authenticate()
    firewall_token = vault_session.retrieve_firewall_secret()
    logging.info("Retrieving token")
    if firewall_token:
        logging.info("Retrieved firewall token")
        addr = input("Firewall IP/FQDN-> ")
        port = int(input("Firewall interface port-> "))
        firewall_session = FortigateMethods(addr, port, firewall_token)
        valid_token = firewall_session.check_token_validity()
        if valid_token:
            clients = firewall_session.retrieve_wireless_clients()
            for client in range (0, len(clients)):
                logging.info("{0} : {1} : {2}".format(client,clients[client]['mac'],clients[client]['ip']))
            selection = int(input("Enter the index of the target MAC address"))
            selection = clients[selection]['mac']
            logging.info("Selected: {0}".format(selection))
            firewall_session.disassociate_wireless_client(selection)
    else:
        logging.error("Failed to retrieve firewall token")